### Loading....

In [1]:
import pandas as pd 
import numpy as np
import json
from pandas.io.json import json_normalize
from tqdm import  tqdm_notebook

In [2]:
path = "C:/Users/hitesh/Learning/Downloads/Omdena AI/drive-download-20190817T143251Z-001/"

In [3]:
filename = "data/idecoNov30_raw_decoded.csv"

In [4]:
data = pd.read_csv(path+filename)

C:\Users\hitesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (3,19,25,26,51,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.drop(columns= ['tld','geo_point','fulldomain'], axis= 1,inplace= True)

In [5]:
data.head()

,decahose_entry.actor.displayName,decahose_entry.actor.favoritesCount,decahose_entry.actor.followersCount,decahose_entry.actor.friendsCount,decahose_entry.actor.id,decahose_entry.actor.image,decahose_entry.actor.languages,decahose_entry.actor.link,decahose_entry.actor.links,decahose_entry.actor.listedCount,...,decahose_entry.twitter_quoted_status.twitter_entities.hashtags,decahose_entry.twitter_quoted_status.twitter_entities.media,decahose_entry.twitter_quoted_status.twitter_entities.symbols,decahose_entry.twitter_quoted_status.twitter_entities.urls,decahose_entry.twitter_quoted_status.twitter_entities.user_mentions,decahose_entry.twitter_quoted_status.twitter_extended_entities.media,decahose_entry.twitter_quoted_status.twitter_filter_level,decahose_entry.twitter_quoted_status.twitter_lang,decahose_entry.twitter_quoted_status.verb,decahose_entry.verb
0,😜just me filch🤔,88386.0,5540.0,5919,id:twitter.com:791987130,https://pbs.twimg.com/profile_images/774937954...,['en'],http://www.twitter.com/1FilchFelonious,"[{'href': None, 'rel': 'me'}]",136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,share
1,Falcons (We-Suck)😔,35553.0,1100.0,917,id:twitter.com:2308675470,https://pbs.twimg.com/profile_images/895077537...,['en'],http://www.twitter.com/MJSavage_,"[{'href': None, 'rel': 'me'}]",9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
2,Miguel😈,14988.0,334.0,395,id:twitter.com:721147211195621376,https://pbs.twimg.com/profile_images/105897053...,['en'],http://www.twitter.com/yung_jhonny,"[{'href': None, 'rel': 'me'}]",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
3,ㅤ,256.0,20.0,15,id:twitter.com:931325883784232960,https://pbs.twimg.com/profile_images/931327083...,['en'],http://www.twitter.com/scammmmmer2,"[{'href': None, 'rel': 'me'}]",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,share
4,Κ Ε N D Ο,653.0,215.0,208,id:twitter.com:884013342301585408,https://pbs.twimg.com/profile_images/105989592...,['en'],http://www.twitter.com/ken_mazing,[{'href': 'https://buy.freshprints.com/collect...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,share


In [17]:
data["decahose_entry."][1]

nan

In [15]:
import re
import numpy as np

def clean_tweet2(input_text, pattern):
    words = re.findall(pattern, input_text)
    return words

In [16]:
clean_tweet2(data["decahose_entry.body"][1], "#[\w]*")


[]

### Cleaning Raw JSON 

In [11]:
pd.set_option('display.max_colwidth', -1)

In [11]:
""" Convert Raw file back to Json file for easy extraction"""

def json_file(raw_data):
    json_acceptable_string = raw_data.replace("'", r"\\")
    return json.loads(json_acceptable_string)

In [12]:
def process(indict, current_key=None, outerdict=None):
    if outerdict is None:
        outerdict = {}
    for key, value in indict.items():
        newkey = current_key + '_' + key if current_key else key
        if type(value) is not dict:
            outerdict[newkey] = value
        else:
            process(value, current_key=newkey, outerdict=outerdict)
    return outerdict

In [13]:
def raw_dataframe(df):
    df_return = []   
    for i in tqdm_notebook(range(len(df))):
        raw_data = df.raw[i]
        json_raw_data = json_file(raw_data)
        processed_json= process(json_raw_data)
        df_add = pd.DataFrame(list(processed_json.items()), columns=["Fields", 'DateValue'])
        df_return.append(df_add)
    return df_return

In [14]:
df5 = raw_dataframe(data)

### Cleaning Tweets

In [ ]:
"""      Different Approach 

!pip install tweet-preprocessor # This library removes URL,Mention,Hashtags,
Reserved_words,Emojis,Smiley,Numbers from tweets

import preprocessor as p
#p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER) # it is possible to opt out certain processes
p.clean('Omdena is a great platform #awesome 👍 https://github.com/s/preprocessor @wura')

import preprocessor as p
import re
def use_tweet_preprocessor(tweet):
    """A python function that returns clean tweets, main functionalty based on https://github.com/s/preprocessor

    Args:
        tweet: tweet to be cleaned
        
    Returns:
        cleaned tweet based on selected p.set_options and defined regrex """
    
    p.set_options(p.OPT.EMOJI,p.OPT.HASHTAG,p.OPT.SMILEY) #retains emojis,hashtags and smileys in the cleaned tweets
    clean_tweet = p.clean(tweet)
    clean_tweet = re.sub('@[^\s]+', '',clean_tweet) #to remove all words that starts with '@' using regrex
    return clean_tweet

ni_decoded_df['clean_content'] = ni_decoded_df['content'].apply(use_tweet_preprocessor)

"""

In [ ]:
import re
import numpy as np

def clean_tweet(input_text,pattern, replace):
    words = re.findall(pattern, input_text)
    for i in words:
        input_text = re.sub(i,replace,input_text)
    return input_text  

In [ ]:
data['cleaned_tweet'] = np.vectorize(clean_tweet)(data['content'], "@[\w]*","twitter_user")

In [ ]:
data['cleaned_tweet'][:10]

### Exploratory Data Analysis to inspect the inspect the distribution of the tweets with keywords over the time of the day and week. Also, a simple visualization to see which keywords are used very often.

"""Added two more columns in the dataset (Hour and Weekday(Day Name))
Prepared dataset with occurences of each keywords in the whole dataset and collected the ids of tweets which has the keywords in them
Filtered out the tweets with keywords
Came up with three visualization -- One showing the frequency of occurences of each keyword in the whole dataset -- One showing the distrbution of tweets(with signal keywords) over the time of a day -- One showing the distribution of tweets(with signal keywords) over the day in a week """

In [ ]:
#Added two new columns
data['hour'] = data['timestamp'].dt.hour
data['day_name'] = data['timestamp'].dt.day_name()

In [ ]:
signal_keywords = pd.read_csv(path + 'signal_keywords')

In [ ]:
#Converted the signal_keywords to list
list_keywords = signal_keywords.Keyword.unique().tolist()

In [ ]:
#Code to find out the occurences of each keywords in the whole dataset and collect the ids of tweets which has the keywords in them

from collections import defaultdict
tweet_dict = dict(zip(ni_decoded_df.id, ni_decoded_df.cleaned_tweet))
dict_keyword = defaultdict(int)

list_of_ids_with_keywords = []

for id,tweet in tweet_dict.items():
    has_keyword = 0
    for word in tweet.split():
        if word in list_keywords:
            dict_keyword[word] += 1
            has_keyword += 1
  
    if has_keyword > 0:
        list_of_ids_with_keywords.append(id)

keyword_cat_ocuurence = pd.DataFrame(list(dict_keyword.items()), columns=['Keyword', 'Number of Occurences'])

In [ ]:
#filter out only the tweets with the keywords
df_ids = pd.DataFrame(data = list_of_ids_with_keywords, columns=['id'])

df_tweets_with_keywords_only = pd.merge(ni_decoded_df, df_ids, how='inner')

In [ ]:
#Visualization

import seaborn as sns
from matplotlib import pyplot

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)

ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

plot1 = sns.barplot(ax = ax, x='Keyword', y = 'Number of Occurences', data = keyword_cat_ocuurence)

In [ ]:
# Visualizing at which time of the day are tweets with keywords usually posted
ax = sns.countplot(x="hour", data=df_tweets_with_keywords_only)

In [ ]:
#Visualizing at which day of the weeks are usually the tweets with keywords usually posted

ax = sns.countplot(x="day_name", data=df_tweets_with_keywords_only)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)